In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
SCATTERING_PARAMS = {
    "nx": 1024,
    "x_min": -200.0,
    "x_max": 200.0,
    "hbar": 1.0,
    "mass": 1.0,
    "dt": 0.03,
    "n_steps": 1400,
    "save_every": 8,
    "barrier_height": 1.0,
    "barrier_width": 8.0,
    "x0": -90.0,
    "sigma": 10.0,
    "k0": 1.3,
}

In [ ]:
def simulate_1d_scattering(
    nx=1024,
    x_min=-200.0,
    x_max=200.0,
    hbar=1.0,
    mass=1.0,
    dt=0.03,
    n_steps=1400,
    save_every=8,
    barrier_height=1.0,
    barrier_width=8.0,
    x0=-90.0,
    sigma=10.0,
    k0=1.3,
):
    """1D time-dependent Schrodinger simulation via split-step FFT."""
    x = np.linspace(x_min, x_max, nx, endpoint=False)
    dx = x[1] - x[0]
    k = 2.0 * np.pi * np.fft.fftfreq(nx, d=dx)

    V = np.where(np.abs(x) < barrier_width / 2.0, barrier_height, 0.0)

    psi = np.exp(-((x - x0) ** 2) / (4.0 * sigma**2)) * np.exp(1j * k0 * x)
    psi /= np.sqrt(np.sum(np.abs(psi) ** 2) * dx)
    norm_initial = np.sum(np.abs(psi) ** 2) * dx

    exp_v_half = np.exp(-1j * V * dt / (2.0 * hbar))
    exp_t = np.exp(-1j * (hbar * k**2 / (2.0 * mass)) * dt)

    snapshots = []
    times = []

    for step in range(n_steps):
        psi = exp_v_half * psi
        psi_k = np.fft.fft(psi)
        psi_k *= exp_t
        psi = np.fft.ifft(psi_k)
        psi = exp_v_half * psi

        if step % save_every == 0:
            snapshots.append(np.abs(psi) ** 2)
            times.append(step * dt)

    left_region = x < -barrier_width / 2.0
    right_region = x > barrier_width / 2.0

    norm_final = np.sum(np.abs(psi) ** 2) * dx
    reflection = np.sum(np.abs(psi[left_region]) ** 2) * dx / norm_final
    transmission = np.sum(np.abs(psi[right_region]) ** 2) * dx / norm_final

    return {
        "x": x,
        "V": V,
        "density_map": np.array(snapshots),
        "times": np.array(times),
        "reflection": reflection,
        "transmission": transmission,
        "norm_drift": abs(norm_final - norm_initial),
    }


result = simulate_1d_scattering(**SCATTERING_PARAMS)
print(f"Reflection ~ {result['reflection']:.3f}")
print(f"Transmission ~ {result['transmission']:.3f}")
print(f"R + T ~ {result['reflection'] + result['transmission']:.3f}")
print(f"Norm drift ~ {result['norm_drift']:.2e}")

In [ ]:
# 1D scattering visualization
x = result["x"]
V = result["V"]
density_map = result["density_map"]
times = result["times"]

snapshot_idx = [0, len(density_map) // 3, 2 * len(density_map) // 3, len(density_map) - 1]

plt.figure(figsize=(10, 5))
for idx in snapshot_idx:
    plt.plot(x, density_map[idx], label=f"t = {times[idx]:.2f}")

v_scale = np.max(V) if np.max(V) > 0 else 1.0
plt.plot(x, V / (v_scale * 8.0), "k--", alpha=0.7, label="Potential (scaled)")
plt.xlim(-150, 120)
plt.ylim(bottom=0)
plt.xlabel("x")
plt.ylabel(r"$|\\psi(x,t)|^2$")
plt.title("Wave Packet Scattering")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 5))
extent = [x[0], x[-1], times[-1], times[0]]
plt.imshow(density_map, aspect="auto", extent=extent, cmap="magma")
plt.colorbar(label=r"$|\\psi|^2$")
plt.xlabel("x")
plt.ylabel("time")
plt.title("Probability Density Evolution")
plt.tight_layout()
plt.show()

In [ ]:
def simulate_two_level(delta=1.0, omega=2.0, dt=0.01, n_steps=3000):
    """Two-level system with Hamiltonian H = (delta/2) sz + (omega/2) sx."""
    sigma_x = np.array([[0.0, 1.0], [1.0, 0.0]], dtype=complex)
    sigma_z = np.array([[1.0, 0.0], [0.0, -1.0]], dtype=complex)

    H = 0.5 * delta * sigma_z + 0.5 * omega * sigma_x
    evals, evecs = np.linalg.eigh(H)
    U_dt = evecs @ np.diag(np.exp(-1j * evals * dt)) @ evecs.conj().T

    psi = np.array([1.0 + 0j, 0.0 + 0j])
    t = np.arange(n_steps) * dt
    p0 = np.empty(n_steps)
    p1 = np.empty(n_steps)

    for i in range(n_steps):
        p0[i] = np.abs(psi[0]) ** 2
        p1[i] = np.abs(psi[1]) ** 2
        psi = U_dt @ psi
        psi /= np.linalg.norm(psi)

    return t, p0, p1


t2, p0, p1 = simulate_two_level()
plt.figure(figsize=(9, 4))
plt.plot(t2, p0, label="P(|0>)")
plt.plot(t2, p1, label="P(|1>)")
plt.xlabel("time")
plt.ylabel("population")
plt.title("Two-Level Quantum Oscillation")
plt.legend()
plt.tight_layout()
plt.show()